This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'wDuaQqc5fyXB6FDcaGx-'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [50]:
# First, import the relevant modules
import requests
from pprint import pprint
import json
import numpy as np

In [39]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url="https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31"+API_KEY
r = requests.get(url)

In [94]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_json=json.loads(r.text)
data = r.json()
dates=[]
opens=[]
highs=[]
lows=[]
closes=[]
volumes=[]

#pprint(data_json)
print (data['dataset_data']['column_names'])
for c in data['dataset_data']['data']:
    dates.append(c[0])
    opens.append(float(c[1]) if c[1] is not None else float('NaN'))
    highs.append(float(c[2]) if c[2] is not None else float('NaN'))
    lows.append(float(c[3]) if c[3] is not None else float('NaN'))
    closes.append(float(c[4]) if c[4] is not None else float('NaN'))
    volumes.append(int(c[6]) if c[6] is not None else int('NaN'))
 
            
     

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [124]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return  np.isnan(y), lambda z: z.nonzero()[0]

In [134]:
def interpolate_array(y):
    
    nans, x= nan_helper(y) 
     
    y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    return y

opens=interpolate_array(np.array(opens))
closes=interpolate_array(np.array(closes))
lows=interpolate_array(np.array(lows))
highs=interpolate_array(np.array(highs))
volumes=interpolate_array(np.array(volumes))
changes = highs-lows

twodaychanges=[]
for i in range(len(dates)-1):
    twodaychanges.append((max((highs[i],highs[i+1]))-min((lows[i],lows[i]))))


In [150]:
print('Highest opening price:',opens.max())
print('Lowest opening price:',opens.min()) 
print('Largest change in a day:', changes.max())
print('Largest two day price change:',np.array(twodaychanges).max())
print('Average volume:', int(volumes.mean()))
print('The median volume during the year:', int(np.median(volumes)))

Highest opening price: 53.11
Lowest opening price: 34.0
Largest change in a day: 2.81
Largest two day price change: 4.3
Average volume: 89124
The median volume during the year: 76286


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)